In [63]:
from types import SimpleNamespace

import numpy as np
from scipy import optimize
import scipy.stats as stats

import pandas as pd
import matplotlib.pyplot as plt


class HouseholdOptimizationClass:

    def __init__(self):
        """ definition of parameters """

        # a. gen namespaces
        par = self.par = SimpleNamespace()  # exogenous
        sol = self.sol = SimpleNamespace()  # endogenous

        # b. preferences
        par.rho = 2.0
        par.nu = 0.001
        par.epsilon = 1.0
        par.omega = 0.5

        # c. household production
        par.alpha = 0.5
        par.sigma = 1.0

        # d. wages
        par.wM = 1.0
        par.wF = 1.0
        par.wF_vec = np.linspace(0.8, 1.2, 5)  # different values for wF (Q2)

        """
        # e. targets
        par.beta0_target = 0.4  # targets from Siminski & Yetsenga for Q4
        par.beta1_target = -0.1  # targets from Siminski & Yetsenga for Q4
        """

        # f. solution
        sol.LM_vec = np.zeros(par.wF_vec.size)
        sol.HM_vec = np.zeros(par.wF_vec.size)
        sol.LF_vec = np.zeros(par.wF_vec.size)
        sol.HF_vec = np.zeros(par.wF_vec.size)

        sol.beta0 = np.nan
        sol.beta1 = np.nan

    def utility(self, LM, HM, LF, HF):
        """ calculating utility 
        args: self, LM, HM, LF, HF
        tilføj?"""

        par = self.par
        sol = self.sol

        # a. consumption of market goods
        C = par.wM*LM + par.wF*LF

        # b. home production
        try:
            if par.sigma == 0:
                H = min(HM, HF)
            elif par.sigma == 1:
                H = HM**(1-par.alpha)*HF**par.alpha
            else:
                H = ((1-par.alpha)*HM**((par.sigma-1)/(par.sigma + 1e-8)) \
                        + par.alpha*HF**((par.sigma-1)/(par.sigma + 1e-8)))**((par.sigma)/(par.sigma-1 + 1e-8))
        except:
            print(f"alpha = {par.alpha}")
            print(f"sigma = {par.sigma}")
            print(f"HM = {HM}")
            print(f"HF = {HF}")
            raise Exception('Error calculating H!')

        # c. total consumption utility
        Q = C**par.omega*H**(1-par.omega)
        utility = np.fmax(Q, 1e-8)**(1-par.rho)/(1-par.rho + 1e-8)

        # d. disutility of work
        epsilon_ = 1+1/(par.epsilon + 1e-8)  # to shorten the function
        TM = LM+HM
        TF = LF+HF
        disutility = par.nu*(TM**epsilon_/(epsilon_ + 1e-8) +
                             TF**epsilon_/(epsilon_ + 1e-8))

        return utility - disutility

    def solve_discrete(self, do_print=False):
        """ solve model discretely """

        par = self.par
        sol = self.sol
        opt = SimpleNamespace()

        # a. possible choices of labor
        x = np.linspace(0, 24, 49)
        LM, HM, LF, HF = np.meshgrid(x, x, x, x)  # all combinations

        LM = LM.ravel()  # unravels LM from the meshgrid?
        HM = HM.ravel()
        LF = LF.ravel()
        HF = HF.ravel()

        # b. utility
        u = self.utility(LM, HM, LF, HF)

        # c. if T > 24 return minus infinity (constraint broken)
        I = (LM+HM > 24) | (LF+HF > 24)
        u[I] = -np.inf

        # d. find maximizing argument for endogenous variables
        j = np.argmax(u)

        opt.LM = LM[j]  # from unravel function ?
        opt.HM = HM[j]
        opt.LF = LF[j]
        opt.HF = HF[j]

        # e. print ? does the dictionary need to be pre-defined ?
        if do_print:
            for k, v in opt.__dict__.items():
                print(f'{k} = {v:6.4f}')

        return opt

    def solve_cont(self, do_print=False):

        par = self.par
        sol = self.sol
        opt = SimpleNamespace()

        def objective(x):
            LM, HM, LF, HF = x
            if HM < 0.0:
                print(f"HM = {HM}")
                print(f"alpha = {par.alpha}")
                print(f"sigma = {par.sigma}")
                raise Exception('HM is below zero') 
            return -self.utility(LM, HM, LF, HF)

        # d. constraints and bounds: if T > 24 return minus infinity (constraint broken)
        """def budget_constraint(LM, HM, LF, HF): return (
            LM+HM > 24) | (LF+HF > 24)  # violated if negative
        constraints = ({'type': 'ineq', 'fun': budget_constraint})
        bounds = ((1e-8, 24-1e-8), (1e-8, 24-1e-8),
                  (1e-8, 24-1e-8), (1e-8, 24-1e-8))"""

        # c. call solver
        x0 = [2, 2, 2, 2]
        # method='SLSQP',bounds=bounds,constraints=constraints
        # TODO: Add bounds! (HM cannot be negative)
        result = optimize.minimize(objective, x0)

        # d. unpack variables
        LM = result.x[0]
        LF = result.x[1]
        HM = result.x[2]
        HF = result.x[3]

        # Assert that we get a valid solution
        if HM < 0.0:
            print(f"alpha = {par.alpha}")
            print(f"sigma = {par.sigma}")
            print(f"result = {result}")
            raise Exception('HM is below zero') 

        return LM, LF, HM, HF
        # return result

    """
    def ratios(self):
        sol = self.sol
        par = self.par

        sol.HM_wage_vec = ()
        sol.HF_wage_vec = ()
        sol.solution_wage = []
        par.wF_list = (0.8, 0.9, 1.0, 1.1, 1.2)


        # b. for loop
        for wages in par.wF_list:
            par.wF = wages
            sol.solution_wage.append(self.solve_cont())
            
        #c. extracting results
        sol.HF_wage_vec = [ns[3] for ns in sol.solution_wage]
        sol.HM_wage_vec = [ns[2] for ns in sol.solution_wage]

        sol.ratio_H = [np.log(a/b) for a, b in zip(sol.HF_wage_vec, sol.HM_wage_vec)]
        sol.ratio_w = np.log(par.wF_list)    
    """

    def get_ratios(self):
        """
        Returns: 
           log(HF/HM)
           log(wf/wm)
        """
        sol = self.sol
        par = self.par

        sol.HM_wage_vec = []
        sol.HF_wage_vec = []
        # sol.solution_wage = []
        par.wF_list = (0.8, 0.9, 1.0, 1.1, 1.2)

        # b. for loop
        for wages in par.wF_list:
            par.wF = wages
            _, _, HM, HF = self.solve_cont()
            #sol.solution_wage.append()
            sol.HM_wage_vec.append(HM)
            sol.HF_wage_vec.append(HF)

        # c. extracting results
        #sol.HF_wage_vec = [ns[3] for ns in sol.solution_wage]
        #sol.HM_wage_vec = [ns[2] for ns in sol.solution_wage]

        ratio_H = [np.log(a/b)
                   for a, b in zip(sol.HF_wage_vec, sol.HM_wage_vec)]
        ratio_w = np.log(par.wF_list)

        return ratio_H, ratio_w

    def regress(self):
        sol = self.sol
        par = self.par

        self.ratios()
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            sol.ratio_w, sol.ratio_H)
        sol.beta0 = intercept
        sol.beta1 = slope


In [55]:
import numpy as np
from   mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy import optimize

In [64]:
def objective_fnc(parameters):
    # Unpack parameters
    alpha, sigma = parameters
    
    # Initialize class instance
    func = HouseholdOptimizationClass()

    # Set parametes of func
    func.par.alpha = alpha
    func.par.sigma = sigma

    # Get regression equation parts
    ratio_H, ratio_w = func.get_ratios()

    # Perform regression
    slope, intercept, _, _, _ = stats.linregress(ratio_w, ratio_H)
    # print(slope, intercept)

    # Calculate loss
    beta0_target = 0.4
    beta0 = intercept
    beta1_target = -0.1
    beta1 = slope
    loss = (beta0_target - beta0)**2 + (beta1_target - beta1)**2 
    return loss


# Advanced debug (will stop program instead of make runtimewarning):
np.seterr(all='raise')

""""""
# Perform optimization
x1 = (0.5, 0.9)  # Initial guess
optimize_result = optimize.minimize(objective_fnc, x1)
best_alpha, best_sigma = optimize_result.x
print(f"best_alpha = {best_alpha}, best_sigma = {best_sigma}")


HM = -0.807458726955414
alpha = 1.5038603043577363
sigma = 0.7888042746559326


Exception: HM is below zero

In [ ]:
def testabc():
    par = SimpleNamespace()
    par.alpha = 1.5038603043577363
    par.sigma = 0.7888042746559326
    HM = -0.807458726955414
    HF = 7.2495019217185614

    tmp = (par.sigma-1) / (par.sigma + 1e-8)
    print(tmp)

    tmp2 = HM**tmp
    print(tmp2)

    res = ((1-par.alpha)*HM**((par.sigma-1)/(par.sigma + 1e-8)) + par.alpha*HF**((par.sigma-1)/(par.sigma + 1e-8)))**((par.sigma)/(par.sigma-1 + 1e-8))
    print(res)

testabc()

-0.26774160517673723
(0.7059017430282513-0.7893275940014801j)
(-3.4651610357967413-3.1285196351436073j)


In [ ]:
# Debug objective_fnc
debug_x = (0.1, 1.0)
debug_result = objective_fnc(debug_x)
print(debug_result)

-1.1404532414957902 -2.1849946053495777
7.764740057425516
